In [3]:
import json
from datasets import load_dataset
import numpy as np

dataset_id = "bigbio/medhop"

dataset = load_dataset(dataset_id, "medhop_bigbio_qa", trust_remote_code=True)["validation"]
dataset

Dataset({
    features: ['id', 'question_id', 'document_id', 'question', 'type', 'choices', 'context', 'answer'],
    num_rows: 342
})

In [11]:
dataset[0]

{'id': 'MH_dev_0',
 'question_id': 'MH_dev_0',
 'document_id': 'MH_dev_0',
 'question': 'interacts_with DB01171?',
 'type': 'multiple_choice',
 'choices': ['DB00786',
  'DB00863',
  'DB01656',
  'DB02377',
  'DB04844',
  'DB04871',
  'DB05812',
  'DB06101',
  'DB08820'],
 'context': "DB08820 : the first therapy acting on the primary cause of cystic fibrosis . Cystic fibrosis ( CF ) is a life-shortening disorder that affects over 30,000 people in the U.S. and 70,000 worldwide . CF is caused by mutations in the P13569 gene , which codes for the cystic fibrosis transmembrane conductance regulator ( P13569 ) protein . P13569 is a chloride and bicarbonate channel and regulates several ion transporters at the epithelial cell membrane , controlling hydration or ionic composition of epithelial secretions . Management of CF is currently supportive , but recent advances in drug development have focused on therapies that assist mutant P13569 function . In the current review , we summarize the dev

In [4]:
from collections import Counter

n_contexts_for_raft = 50
unique_contexts = Counter(dataset["context"])
count = 0
raft_train_ctx = []
raft_test_ctx = []
for k, v in unique_contexts.items():
  if v > 0 and count < n_contexts_for_raft:
    count += 1
    raft_train_ctx.append(k)
  else:
    raft_test_ctx.append(k)

print(len(raft_test_ctx), len(raft_train_ctx))

#raft_train_ctx = unique_contexts[:100]
#raft_test_ctx = unique_contexts[100:]

292 50


In [7]:
test_dataset = dataset.filter(lambda x: x["context"] in raft_test_ctx)
test_dataset

Dataset({
    features: ['id', 'question_id', 'document_id', 'question', 'type', 'choices', 'context', 'answer'],
    num_rows: 292
})

In [6]:
from datasets import Dataset

test_dataset_unique_ctx = test_dataset.to_pandas()
test_dataset_unique_ctx = test_dataset_unique_ctx.groupby("context").first().reset_index()
test_dataset_unique_ctx = Dataset.from_pandas(test_dataset_unique_ctx)
test_dataset_unique_ctx

Dataset({
    features: ['context', 'id', 'question_id', 'document_id', 'question', 'type', 'choices', 'answer'],
    num_rows: 292
})

In [8]:
# Dataset for evaluation raft
uploaded_dataset_id = "phatvo/medhop-raft-test"
test_dataset.push_to_hub(uploaded_dataset_id, split="test", private=True)
#test_dataset_unique_ctx.push_to_hub(uploaded_dataset_id, split="test_unique_ctx", private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.11s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/phatvo/medhop-raft-test/commit/c19f4524f39b431c761f85fc928804cd42657ae9', commit_message='Upload dataset', commit_description='', oid='c19f4524f39b431c761f85fc928804cd42657ae9', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
def save_context_to_file(data:list, filepath):
  with open(filepath, "w") as f:
    _data = [each.replace("\n", "") for each in data]
    f.write("\n".join(_data) + "\n")

save_context_to_file(raft_train_ctx, "../data/context_for_generate_medhop.txt")

In [10]:
with open("../data/context_for_generate_medhop.txt", "r") as f:
  print(len(f.readlines()))

50


In [ ]:
"""

python3 raft.py \
    --datapath "data/context_for_generate_medhop.txt" \
    --output "tmp/medhop-test-50-raft" \
    --distractors 3 \
    --doctype txt \
    --chunk_size 512 \
    --questions 1 \
    --openai_key $OPENAI_KEY \
    --completion_model gpt-4o \
    --splitter breakline --p 0.95

"""

In [ ]:
# $env:OPENAI_KEY=

In [12]:
from datasets import load_dataset

ds = load_dataset("json", data_files="../tmp/medhop-test-50-raft.jsonl", )
ds = ds.map(lambda x: {"text": f"{x['instruction']}\nCoT Answer: {x['cot_answer']}"})
ds

Generating train split: 57 examples [00:00, 576.65 examples/s]
Map: 100%|██████████| 57/57 [00:00<00:00, 431.42 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 57
    })
})

In [13]:
ds.push_to_hub("phatvo/medhop-50-raft")

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.99s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/phatvo/medhop-50-raft/commit/33c66e7270356278b521daecb9bdf9e61cec0e70', commit_message='Upload dataset', commit_description='', oid='33c66e7270356278b521daecb9bdf9e61cec0e70', pr_url=None, pr_revision=None, pr_num=None)